In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
import sys
from pathlib import Path

if "workding_dir" not in locals():
    workding_dir = str(Path.cwd().parent)
os.chdir(workding_dir)
sys.path.append(workding_dir)
print("working dir:", workding_dir)

working dir: /home/inflaton/code/CrediNews


In [3]:
from dotenv import find_dotenv, load_dotenv

found_dotenv = find_dotenv(".env")

if len(found_dotenv) == 0:
    found_dotenv = find_dotenv(".env.example")
print(f"loading env vars from: {found_dotenv}")
load_dotenv(found_dotenv, override=True)

loading env vars from: /home/inflaton/code/CrediNews/.env


True

In [4]:
from llm_experiements.utils import *
from datasets import load_dataset

dataset = load_dataset(
    "csv",
    data_files={
        "test": "dataset/scrapped_news.csv",
    },
)
dataset

2025-02-23 07:09:58.100866: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-02-23 07:09:58.110532: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1740265798.122568 3544182 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1740265798.126119 3544182 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-02-23 07:09:58.137908: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instr

Generating test split: 0 examples [00:00, ? examples/s]

DatasetDict({
    test: Dataset({
        features: ['source', 'full_content', 'processed_full_content'],
        num_rows: 1275
    })
})

In [5]:
%%time

model_name = "./results/DistilBERT-combined"
pred_combined_model_val_data = evaluate_fine_tuned_llm(model_name, eval_dataset=dataset["test"])

Map:   0%|          | 0/1275 [00:00<?, ? examples/s]

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: inflaton-sg (inflaton-ai) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin



🔹 #Predictions:
1275
CPU times: user 6.89 s, sys: 710 ms, total: 7.6 s
Wall time: 13.3 s


In [6]:
def analyze_predictions(dataset, predictions):
    data_copy = dataset["test"].to_pandas()

    # Add predictions to scraped data
    data_copy["predicted_label"] = predictions

    # Print predictions by source
    print("\nPredictions by Source:")
    source_predictions = (
        data_copy.groupby("source")["predicted_label"].value_counts().unstack()
    )
    source_predictions.columns = ["Predicted Real", "Predicted Fake"]
    print(source_predictions)

    # Calculate percentage of fake news by source
    fake_percentages = data_copy.groupby("source")["predicted_label"].mean() * 100
    print("\nPercentage of Fake News by Source:")
    print(fake_percentages.sort_values(ascending=False))

analyze_predictions(dataset, pred_combined_model_val_data)


Predictions by Source:
                  Predicted Real  Predicted Fake
source                                          
AP                           118              20
BBC                           51              20
Breitbart                    127              47
CNN                           80              18
Guardian                      43              45
NPR                           63              14
Natural News                  55              73
News Max                      95              94
The Daily Caller              67              91
Zerohedge                     44             110

Percentage of Fake News by Source:
source
Zerohedge           71.428571
The Daily Caller    57.594937
Natural News        57.031250
Guardian            51.136364
News Max            49.735450
BBC                 28.169014
Breitbart           27.011494
CNN                 18.367347
NPR                 18.181818
AP                  14.492754
Name: predicted_label, dtype: float64


In [8]:
%%time

model_name = "./results/DistilBERT-original"
pred_original_model_val_data = evaluate_fine_tuned_llm(model_name, eval_dataset=dataset["test"])


🔹 #Predictions:
1275
CPU times: user 4.51 s, sys: 43.3 ms, total: 4.55 s
Wall time: 9.84 s


In [9]:
analyze_predictions(dataset, pred_original_model_val_data)


Predictions by Source:
                  Predicted Real  Predicted Fake
source                                          
AP                            90              48
BBC                           51              20
Breitbart                    105              69
CNN                           73              25
Guardian                      41              47
NPR                           59              18
Natural News                  66              62
News Max                     163              26
The Daily Caller              80              78
Zerohedge                     42             112

Percentage of Fake News by Source:
source
Zerohedge           72.727273
Guardian            53.409091
The Daily Caller    49.367089
Natural News        48.437500
Breitbart           39.655172
AP                  34.782609
BBC                 28.169014
CNN                 25.510204
NPR                 23.376623
News Max            13.756614
Name: predicted_label, dtype: float64
